https://satijalab.org/seurat/archive/v3.1/atacseq_integration_vignette.html![image.png](attachment:image.png)

https://satijalab.org/seurat/archive/v3.0/atacseq_integration_vignette.html
Note: this data isn't matched.
Data from https://satijalab.org/signac/articles/pbmc_multiomic.html

In [1]:
library(Seurat)
library(ggplot2)
library(patchwork)

In [2]:
data <- readRDS("../../data/pbmc/data_list.rds")
peaks <- data$Peaks
genes = data[['Gene Expression']]

In [3]:
atac <- readRDS("../../Seurat3/pbmc/atac.rds")

In [5]:
activity.matrix = atac[['ACTIVITY']]

In [6]:
rna_count = data[["Gene Expression"]]
shared_genes = intersect(rownames(rna_count), rownames(activity.matrix))
colnames(rna_count) = paste0("rna.", colnames(rna_count))
colnames(activity.matrix) = paste0("atac.", colnames(activity.matrix))

In [8]:
cat_count_matrix = cbind(rna_count[shared_genes,], activity.matrix[shared_genes,])

In [11]:
str(cat_count_matrix)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:48255478] 14 19 34 42 43 49 58 60 70 78 ...
  ..@ p       : int [1:22649] 0 2014 3201 4992 5472 6892 7715 9625 10661 12582 ...
  ..@ Dim     : int [1:2] 11045 22648
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:11045] "AP006222.2" "LINC01128" "AL645608.2" "SAMD11" ...
  .. ..$ : chr [1:22648] "rna.AAACAGCCAAGGAATC-1" "rna.AAACAGCCAATCCCTT-1" "rna.AAACAGCCAATGCGCT-1" "rna.AAACAGCCACACTAAT-1" ...
  ..@ x       : num [1:48255478] 1 1 1 1 1 1 1 6 3 3 ...
  ..@ factors : list()


In [10]:
cat_count_matrix <- cat_count_matrix[-which(apply(cat_count_matrix, 1, var) == 0),]

In [ ]:
cat_mat.pca = prcomp(t(as.matrix(cat_count_matrix)), center = T, scale. = T, rank. = 20)

## Incorporate cell type

In [ ]:
meta <- read.table("../../data/pbmc/cell_type.txt", stringsAsFactors = F, head = T)

In [ ]:
str(meta)

In [ ]:
meta.ft = c(meta$cluster, meta$cluster)
names(meta.ft) = c(paste0("rna.", meta$bc), paste0("atac.", meta$bc))

In [ ]:
embedding = as.data.frame(cat_mat.pca$x[,1:20])
stopifnot(rownames(embedding) != embedding$bc)
embedding$cell_type = factor(meta.ft[rownames(embedding)])

In [ ]:
embedding$batch = as.numeric(factor(sapply(strsplit(rownames(embedding), split = ".", fixed = T), function(s) paste0(s[2:length(s)], collapse = "."))))

In [ ]:
str(embedding)

In [28]:
write.table(embedding, "../../Raw/pbmc/integrated_embedding.txt", quote = F, sep = "\t", row.names = T, col.names = T)